### Fake News Project

By: Mateo Anusic, Emil Thorlund, Lucas A. Rosing, Victor Bergien

### Task #1

- Tokenize the text
- Remove stopwords and compute the size of the vocabulary.
- Compute the reduction rate of the vocabulary size after removing stopwords.
- Remove word variations with stemming and compute the size of the vocabulary.
- Compute the reduction rate of the vocabulary size after stemming.

Describe which procedures (and which libraries) you used and why they are appropriate.

In [1]:
# Importing the libraries

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pandas as pd
import csv

# Importing the dataset
data = pd.read_csv('raw_shuffle_data_4GB.csv')

# Prints the first 5 rows of the dataset for a quick look
data.head()

/var/folders/nm/_dj8kh1j5hx4wbvcdmw7smmh0000gn/T/ipykernel_9687/575901525.py:12: DtypeWarning: Columns (0,1,11,13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('raw_shuffle_data_4GB.csv')


,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN
1,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN
2,9,15,blackagendareport.com,unreliable,https://blackagendareport.com/zambia-must-clar...,Zambia Must Clarify Whether It Will Host Israe...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Zambia Must Clarify Whether It Will Host Israe...,"Glen Ford, Bar Executive Editor, Margaret Kimb...",NaN,[''],“Israel since 1948 has collaborated with the s...,NaN,NaN,NaN
3,24,31,theshovel.com.au,satire,http://www.theshovel.com.au/tag/australian-pol...,Senators from the Australian Greens will be st...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Australian political satire – The Shovel,The Shovel,NaN,[''],NaN,"5, 3, 6, 4, 2",NaN,NaN
4,30,37,beforeitsnews.com,fake,http://beforeitsnews.com/arts/2018/01/lost-wor...,"Lost Words, Hidden Words, Otters, Banks and Bo...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"Lost Words, Hidden Words, Otters, Banks and Books",Jackie Morris Artist,NaN,[''],NaN,NaN,NaN,NaN


In [3]:
data2 = data.drop(columns = ["Unnamed: 0", "meta_description", "keywords", "tags", "summary", "meta_keywords", "authors", "updated_at",  "scraped_at",  "inserted_at", "url","id", "domain", "source"], axis=1)

In [4]:
print(data2.head())


         type                                            content  \
0       rumor  Life is an illusion, at least on a quantum lev...   
1        hate  The White House has decided to quietly withdra...   
2  unreliable  Zambia Must Clarify Whether It Will Host Israe...   
3      satire  Senators from the Australian Greens will be st...   
4        fake  Lost Words, Hidden Words, Otters, Banks and Bo...   

                                               title  
0  Is life an ILLUSION? Researchers prove 'realit...  
1  MORE WINNING! Israeli intelligence source, DEB...  
2  Zambia Must Clarify Whether It Will Host Israe...  
3           Australian political satire – The Shovel  
4  Lost Words, Hidden Words, Otters, Banks and Books  


In [5]:
# Creates a subset of the dataset with only the columns we need
data4 = data2.dropna(subset=['content', 'title', 'type'])

# Prints the first 5 rows of the dataset for a quick look
print(data4) 

# Removes the rows with unknown or None values
data4 = data4[(data4 != 'unknown') & (data4 != 'None')].dropna()

               type                                            content  \
0             rumor  Life is an illusion, at least on a quantum lev...   
1              hate  The White House has decided to quietly withdra...   
2        unreliable  Zambia Must Clarify Whether It Will Host Israe...   
3            satire  Senators from the Australian Greens will be st...   
4              fake  Lost Words, Hidden Words, Otters, Banks and Bo...   
...             ...                                                ...   
1251821    reliable  37A: Here’s another example of “start” or “beg...   
1251822    reliable  The letter said the Justice Department’s initi...   
1251823    reliable  Photo\n\nSAN FRANCISCO — Snap finally snapped ...   
1251824    reliable  Idaho Stripped Climate Change From School Guid...   
1251825    reliable  Housing Authority residents in the audience we...   

                                                     title  
0        Is life an ILLUSION? Researchers prove 'r

In [6]:
# Creates a function which simplifies the type column
# This allows for an easier classification
def modify_type(x):
    if x == 'fake':
        return 0
    elif x == 'reliable':
        return 1
    elif x == 'rumor':
        return 2
    elif x == 'satire':
        return 3
    elif x == 'bias':
        return 4
    elif x == 'conspiracy':
        return 5
    elif x == 'junksci':
        return 6
    elif x == 'hate':
        return 7
    elif x == 'clickbait':
        return 8
    elif x == 'unreliable':
        return 9
    elif x == 'political':
        return 10
    else:
        return None   

In [7]:
# Creates a new dataframe with the modified type column
data5 = pd.DataFrame(data4)

# Writes type column to the new dataframe
data5['type'] = data5['type'].apply(modify_type)

# Prints the dataframe to verify the changes
data5 

,type,content,title
0,2,"Life is an illusion, at least on a quantum lev...",Is life an ILLUSION? Researchers prove 'realit...
1,7,The White House has decided to quietly withdra...,"MORE WINNING! Israeli intelligence source, DEB..."
2,9,Zambia Must Clarify Whether It Will Host Israe...,Zambia Must Clarify Whether It Will Host Israe...
3,3,Senators from the Australian Greens will be st...,Australian political satire – The Shovel
4,0,"Lost Words, Hidden Words, Otters, Banks and Bo...","Lost Words, Hidden Words, Otters, Banks and Books"
...,...,...,...
1251821,1,37A: Here’s another example of “start” or “beg...,Shared Beliefs
1251822,1,The letter said the Justice Department’s initi...,Senate Letter Echoes House Republicans’ Accusa...
1251823,1,Photo\n\nSAN FRANCISCO — Snap finally snapped ...,Snap Reverses Slide as User Growth and Revenue...
1251824,1,Idaho Stripped Climate Change From School Guid...,Idaho Stripped Climate Change From School Guid...


In [8]:
# Creates a new dataframe in order to check that there are no 'none' values
data6 = data2.dropna(subset=['type'])

In [9]:
# Import the necessary libraries
import numpy as np

In [10]:
# Sorts the dataframe by the type column
data_by_type = {}

# A for-loop which creates a new dataframe for each type
for i in range(11):
    data_by_type[i] = data5[data5['type'] == i]

data_zero = data_by_type[0]
data_one = data_by_type[1]
data_two = data_by_type[2]
data_three = data_by_type[3]
data_four = data_by_type[4]
data_five = data_by_type[5]
data_six = data_by_type[6]
data_seven = data_by_type[7]
data_eight = data_by_type[8]
data_nine = data_by_type[9]
data_ten = data_by_type[10]


In [11]:
# Prints the dataframe containing only type 1 data
data_one

,type,content,title
7,1,"Facebook/DestinyTheGame Promo image for ""Desti...",'Destiny 2' News: Bungie Addresses Player Comp...
14,1,(Screencap: YouTube/Tidningen Dagen) Annahita ...,"Iranian Christian Convert Leads 1,500 Muslims ..."
34,1,Republican Sen. James Lankford of Oklahoma say...,Who's Afraid of the Big Bad Christian?
72,1,(Screengrab: YouTube / ABC15 Arizona) Evangeli...,Franklin Graham Responds to Trump-Porn Star Al...
73,1,"(Photo: Rocket Republic, Courtesy of ERLC) Rus...","To Build Pro-Life Culture, Christians Must Add..."
...,...,...,...
1251821,1,37A: Here’s another example of “start” or “beg...,Shared Beliefs
1251822,1,The letter said the Justice Department’s initi...,Senate Letter Echoes House Republicans’ Accusa...
1251823,1,Photo\n\nSAN FRANCISCO — Snap finally snapped ...,Snap Reverses Slide as User Growth and Revenue...
1251824,1,Idaho Stripped Climate Change From School Guid...,Idaho Stripped Climate Change From School Guid...


In [12]:
# Create DataFrame for fake data with only zeros in "type" column
data_zero_only = pd.DataFrame({'type': np.zeros(len(data_zero))})
data_one_only = pd.DataFrame({'type': np.ones(len(data_one))})
data_two_only = pd.DataFrame({'type': 2 * np.ones(len(data_two))})
data_three_only = pd.DataFrame({'type': 3 * np.ones(len(data_three))})
data_four_only = pd.DataFrame({'type': 4 * np.ones(len(data_four))})
data_five_only = pd.DataFrame({'type': 5 * np.ones(len(data_five))})
data_six_only = pd.DataFrame({'type': 6 * np.ones(len(data_six))})
data_seven_only = pd.DataFrame({'type': 7 * np.ones(len(data_seven))})
data_eight_only = pd.DataFrame({'type': 8 * np.ones(len(data_eight))})
data_nine_only = pd.DataFrame({'type': 9 * np.ones(len(data_nine))})
data_ten_only = pd.DataFrame({'type': 10 * np.ones(len(data_ten))})



In [13]:
# Prints the dataframe containing only type 5 data
data_five_only

,type
0,5.0
1,5.0
2,5.0
3,5.0
4,5.0
...,...
111650,5.0
111651,5.0
111652,5.0
111653,5.0


In [14]:
# Creates the datasets with type, content and title columns

zero_dataset = data_zero[['type', 'content', 'title']]
one_dataset = data_one[['type', 'content', 'title']]
two_dataset = data_two[['type', 'content', 'title']]
three_dataset = data_three[['type', 'content', 'title']]
four_dataset = data_four[['type', 'content', 'title']]
five_dataset = data_five[['type', 'content', 'title']] 
six_dataset = data_six[['type', 'content', 'title']] 
seven_dataset = data_seven[['type', 'content', 'title']] 
eight_dataset = data_eight[['type', 'content', 'title']] 
nine_dataset = data_nine[['type', 'content', 'title']] 
ten_dataset = data_ten[['type', 'content', 'title']]

five_dataset 

,type,content,title
9,5,Follow the below steps to view photos In one o...,SaySo March 2015
23,5,"Amid Deep Partisan Divide, Pence Breaks Tie to...",Family Security Matters
25,5,Clinton–Obama Emails: The Key to Understanding...,Family Security Matters
31,5,Enter your email address\n\n\n\nI have an acco...,Family Security Matters
33,5,With a number of people now identifying as spi...,How Spirituality Could Be the Key to Coping Wi...
...,...,...,...
975312,5,Editor’s Note: Putting people in jail for non-...,Violent Rapists Set Free as the State Makes Ro...
975313,5,Residents of a large Houston neighborhood grew...,Desperate Citizens Hire Private Security to Fi...
975324,5,"Gee, sure seems like the Dems might be slightl...","Scared of Something? DNC Erects 8-Foot High, 4..."
975329,5,At a John Birch Society council dinner speech ...,John Birch Founder Predicted Steps to America’...


In [15]:
# Concatenates the datasets into one dataset
data7 = pd.concat([zero_dataset,
                    one_dataset,
                    two_dataset,
                    three_dataset,
                    four_dataset,
                    five_dataset,
                    six_dataset,
                    seven_dataset,
                    eight_dataset,
                    nine_dataset,
                    ten_dataset
                    ], axis=0)

# Prints the new dataset
data7

,type,content,title
4,0,"Lost Words, Hidden Words, Otters, Banks and Bo...","Lost Words, Hidden Words, Otters, Banks and Books"
5,0,Poll: Calls for War From Israelis and Palestin...,Poll: Calls for War From Israelis and Palestin...
61,0,American Airlines quarterly profits hit by hig...,American Airlines quarterly profits hit by hig...
67,0,Despite the claim of Iranian Revolutionary Gua...,"After 8 Years with Obama, Iran Protesters Reve..."
71,0,Zebrato by Arshad Husain Siddiqui\n\nHeadline:...,Zebrato by Arshad Husain Siddiqui
...,...,...,...
975330,10,This is the fifth installment in an ongoing se...,VIDEO: Maine Residents Defend Voting Rights As...
975331,10,"On Thursday, the Mississippi House passed a 20...",Mississippi Advances A Stringent Abortion Ban ...
975333,10,"Last week, Sen. Elizabeth Warren (D-MA) gained...",Democratic Senator Presses Regulators On Why B...
975334,10,India’s Assam state is the largest tea-produci...,Coffee Or Tea? Something Hiding In Your Trash ...


In [16]:
# Removes the rows with unknown or None values

data7.loc[data5['content'] == " ", 'content']= np.nan
data7.loc[data5['content'] == "  ", 'content']= np.nan

In [17]:
# Removes the rows with unknown or None values
data8 = data7.dropna()

In [18]:
# Prints the new dataset
data8.head()

,type,content,title
4,0,"Lost Words, Hidden Words, Otters, Banks and Bo...","Lost Words, Hidden Words, Otters, Banks and Books"
5,0,Poll: Calls for War From Israelis and Palestin...,Poll: Calls for War From Israelis and Palestin...
61,0,American Airlines quarterly profits hit by hig...,American Airlines quarterly profits hit by hig...
67,0,Despite the claim of Iranian Revolutionary Gua...,"After 8 Years with Obama, Iran Protesters Reve..."
71,0,Zebrato by Arshad Husain Siddiqui\n\nHeadline:...,Zebrato by Arshad Husain Siddiqui


In [19]:
# Renames the content column to con
data9 = data8.rename(columns = {"content":"con"})

In [20]:
# Prints the new dataset
data9

,type,con,title
4,0,"Lost Words, Hidden Words, Otters, Banks and Bo...","Lost Words, Hidden Words, Otters, Banks and Books"
5,0,Poll: Calls for War From Israelis and Palestin...,Poll: Calls for War From Israelis and Palestin...
61,0,American Airlines quarterly profits hit by hig...,American Airlines quarterly profits hit by hig...
67,0,Despite the claim of Iranian Revolutionary Gua...,"After 8 Years with Obama, Iran Protesters Reve..."
71,0,Zebrato by Arshad Husain Siddiqui\n\nHeadline:...,Zebrato by Arshad Husain Siddiqui
...,...,...,...
975330,10,This is the fifth installment in an ongoing se...,VIDEO: Maine Residents Defend Voting Rights As...
975331,10,"On Thursday, the Mississippi House passed a 20...",Mississippi Advances A Stringent Abortion Ban ...
975333,10,"Last week, Sen. Elizabeth Warren (D-MA) gained...",Democratic Senator Presses Regulators On Why B...
975334,10,India’s Assam state is the largest tea-produci...,Coffee Or Tea? Something Hiding In Your Trash ...


In [21]:
# Creates a new dataframe with the modified type column
data10 = data9["con"]

In [22]:
# Squeezes the dataframe
sqeeze = data9.squeeze(1)


In [23]:
# Makes sure that all content rows are less than 1000 characters
data10 = data8["content"].str.slice(start = 0, stop = 1000)

In [24]:
# Creates a new dataframe which is concatenated with the type column
data11 = pd.concat([sqeeze,data10], axis=1).drop(columns='con')

# Prints the new dataset
data11

,type,title,content
4,0,"Lost Words, Hidden Words, Otters, Banks and Books","Lost Words, Hidden Words, Otters, Banks and Bo..."
5,0,Poll: Calls for War From Israelis and Palestin...,Poll: Calls for War From Israelis and Palestin...
61,0,American Airlines quarterly profits hit by hig...,American Airlines quarterly profits hit by hig...
67,0,"After 8 Years with Obama, Iran Protesters Reve...",Despite the claim of Iranian Revolutionary Gua...
71,0,Zebrato by Arshad Husain Siddiqui,Zebrato by Arshad Husain Siddiqui\n\nHeadline:...
...,...,...,...
975330,10,VIDEO: Maine Residents Defend Voting Rights As...,This is the fifth installment in an ongoing se...
975331,10,Mississippi Advances A Stringent Abortion Ban ...,"On Thursday, the Mississippi House passed a 20..."
975333,10,Democratic Senator Presses Regulators On Why B...,"Last week, Sen. Elizabeth Warren (D-MA) gained..."
975334,10,Coffee Or Tea? Something Hiding In Your Trash ...,India’s Assam state is the largest tea-produci...


In [25]:
# Creates the X and y variables which will be used for the classification
x = data11.drop(columns="type") 
y = data11["type"]

In [26]:
# Prints the X variable
x

,title,content
4,"Lost Words, Hidden Words, Otters, Banks and Books","Lost Words, Hidden Words, Otters, Banks and Bo..."
5,Poll: Calls for War From Israelis and Palestin...,Poll: Calls for War From Israelis and Palestin...
61,American Airlines quarterly profits hit by hig...,American Airlines quarterly profits hit by hig...
67,"After 8 Years with Obama, Iran Protesters Reve...",Despite the claim of Iranian Revolutionary Gua...
71,Zebrato by Arshad Husain Siddiqui,Zebrato by Arshad Husain Siddiqui\n\nHeadline:...
...,...,...
975330,VIDEO: Maine Residents Defend Voting Rights As...,This is the fifth installment in an ongoing se...
975331,Mississippi Advances A Stringent Abortion Ban ...,"On Thursday, the Mississippi House passed a 20..."
975333,Democratic Senator Presses Regulators On Why B...,"Last week, Sen. Elizabeth Warren (D-MA) gained..."
975334,Coffee Or Tea? Something Hiding In Your Trash ...,India’s Assam state is the largest tea-produci...


In [27]:
# Prints the y variable
y

4          0
5          0
61         0
67         0
71         0
          ..
975330    10
975331    10
975333    10
975334    10
975335    10
Name: type, Length: 1127671, dtype: int64

In [28]:
# Creates csv files for the X and y variables
x.to_csv("features_multi.csv", index = False)

y.to_csv("labels_multi.csv", index = False)